In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
import hashlib

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30000000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [ ]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'informationretrievalex3'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"

Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd


**Count the frequency of each word in docunment**

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

"""
Count the frequency of each word in docunment
Parameters:
  text: (str) Text of one document
  id: (int) Document id
  withTFnormal (bool) if normalization by leanght of the document
Returns:
  List of tuples: A list of (token, (doc_id, tf))
"""
def word_count(text, id, withTFnormal=False):

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  sizeOfDoc = 0
  word_counts = Counter()

  for token in tokens:
    if token not in all_stopwords:
      word_counts[token] += 1
      sizeOfDoc += 1

  if(not withTFnormal):
    return [(token, (id, count)) for token, count in word_counts.items()]

  return [(token, (id, count/sizeOfDoc)) for token, count in word_counts.items()]



In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))    # Count the frequency of each word in docunment without normalization

**sorted posting list by wiki_id**

In [ ]:
"""
Returns a sorted posting list by wiki_id.
Parameters:
  unsorted_pl: list of tuples - A list of (wiki_id, tf) tuples
Returns:
    list of tuples A sorted posting list.
"""
def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

In [ ]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)   # Sort by document id and then merge by term.
postings_filtered = postings.filter(lambda x: len(x[1])>10)         # Filter out rare words

**Takes a posting list RDD and calculate the df for each token**

In [ ]:
"""
Takes a posting list RDD and calculate the df for each token.
Parameters:
  postings: (RDD) An RDD where each element is a (token, posting_list).
Returns:
  An RDD where each element is a (token, df) pair.
"""
def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

In [ ]:
w2df_dict = (calculate_df(postings_filtered)).collectAsMap()    # Create dict of df

**A function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk**

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

"""
A function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk.
Parameters:
  postings: (RDD) An RDD where each item is a (w, posting_list) pair.
Returns:
  An RDD where each item is a posting locations dictionary for a bucket.
"""
def partition_postings_and_write(postings):

  bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))         # Assign each token to a bucket
  grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)                        # Group the postings by bucket
  posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x))   # Write out each bucket and collect posting locations

  return posting_locations

In [ ]:
# Merge the posting locations into a single dict.
posting_locs_list = partition_postings_and_write(postings).collect()
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)


**Create tf-idf dictionary**

In [ ]:
# Create tf-idf dictionary

N = 1000;

word_TF = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
postings_TF = word_TF.groupByKey().mapValues(reduce_word_counts)
postings_TF_filtered = postings_TF.filter(lambda x: len(x[1])>10)

postings_TF_IDF_filtered = postings_TF_filtered.map(lambda x: (x[0], updated_value_function(x[0],x[1])))

def updated_value_function(term, post):
    return list(map(lambda x: (x[0], x[1]*math.log10(N/w2df_dict[term])), post))

postings_TF_IDF_filtered_dict = postings_TF_IDF_filtered.collectAsMap()

**write the global stats out**

In [ ]:
inverted = InvertedIndex()                            # Create inverted index instance
inverted.posting_locs = super_posting_locs            # Adding the posting locations dictionary to the inverted index
inverted.df = w2df_dict                               # Add the df dictionary to the inverted index
inverted.tfidf = postings_TF_IDF_filtered_dict;       # Add the tfidf dictionary to the inverted index
inverted.write_index('.', 'index')                    # write the global stats out


In [ ]:
inverted_index = InvertedIndex.read_index('.', 'index')

In [ ]:
inverted_index.tfidf["political"]